In [7]:
# import importlib
# importlib.reload(franky)

<module 'franky' (<_frozen_importlib_external.NamespaceLoader object at 0x7e24e8089b10>)>

In [1]:
import franky
import numpy as np
np.set_printoptions(precision=6, suppress=True, linewidth=200)

from scipy.spatial.transform import Rotation

In [2]:
robot = franky.Robot("172.168.0.2", realtime_config=franky.RealtimeConfig.Ignore)

In [3]:
robot.forward_kinematics(np.array([0, 0, 0, 0, 0, 0, 0]))

Affine(t=[0.088 0 0.823], q=[0.92388 0.382683 0 0])

In [4]:
pos = robot.forward_kinematics(np.array(robot.state.q))

In [7]:
%%timeit -r 10 -n 100  q0 = np.array(robot.state.q) + np.random.randn(7) * 0.1
q_result = robot.inverse_kinematics(pos, q0)

156 μs ± 36.5 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [7]:
%%time
q0 = np.array(robot.state.q) + np.random.randn(7) * 0.1
robot.inverse_kinematics(pos, q0)

CPU times: user 30.6 ms, sys: 18.8 ms, total: 49.4 ms
Wall time: 2.02 s


array([-0.077306, -0.30231 ,  0.062147, -1.52047 ,  0.017191,  1.508861,  0.782234])

In [5]:
def quaternion_multiply(q1, q2):
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2

    w = w1 * w2 - x1 * x2 - y1 * y2 - z1 * z2
    x = w1 * x2 + x1 * w2 + y1 * z2 - z1 * y2
    y = w1 * y2 - x1 * z2 + y1 * w2 + z1 * x2
    z = w1 * z2 + x1 * y2 - y1 * x2 + z1 * w2

    return np.array([w, x, y, z])

In [6]:
def q_inv(q):
    q = q.copy()
    q[1:] = -q[1:]
    return q

---

In [30]:
def error(x_cur, target):
    pos_error = x_cur.translation - target.translation
    R_cur = x_cur.matrix[:3,:3]  # Current rotation matrix
    R_target = target.matrix[:3,:3]  # Target rotation matrix
    R_error = R_cur.T @ R_target
    rot = Rotation.from_matrix(R_error)
    rot_vec = rot.as_rotvec()
    rot_vec = -R_cur @ rot_vec
    return np.concatenate([pos_error, rot_vec])

In [9]:
# _q = np.zeros(7)
# _q = np.array([0, np.pi/2, 0, np.pi/2, 0, 0, 0])
_q = np.array([0, 0, 0, 0, 0, 0, 0])

In [10]:
eps = 1e-5
J_num = np.zeros((6, 7))

for i in range(7):
    dq = np.zeros(7)
    dq[i] = eps
    x_plus = error(robot.forward_kinematics(_q + dq), robot.forward_kinematics(_q))
    x_minus = error(robot.forward_kinematics(_q - dq), robot.forward_kinematics(_q))
    J_num[:,i] = (x_plus - x_minus) / (2 * eps)

In [11]:
J_num

array([[ 0.    , -0.49  ,  0.    ,  0.1738,  0.    , -0.2104,  0.    ],
       [-0.088 ,  0.    , -0.088 ,  0.    , -0.088 ,  0.    ,  0.    ],
       [ 0.    ,  0.088 ,  0.    , -0.0055,  0.    , -0.088 ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  1.    ,  0.    , -1.    ,  0.    , -1.    ,  0.    ],
       [ 1.    ,  0.    ,  1.    ,  0.    ,  1.    ,  0.    , -1.    ]])

In [12]:
J = robot.jacobian(_q)
J

array([[ 0.    ,  0.49  ,  0.    , -0.1738,  0.    ,  0.2104,  0.    ],
       [ 0.088 ,  0.    ,  0.088 , -0.    ,  0.088 ,  0.    ,  0.    ],
       [ 0.    , -0.088 ,  0.    ,  0.0055,  0.    ,  0.088 ,  0.    ],
       [ 0.    , -0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  1.    ,  0.    , -1.    , -0.    , -1.    ,  0.    ],
       [ 1.    ,  0.    ,  1.    , -0.    ,  1.    ,  0.    , -1.    ]])

---

In [31]:
def line_search(robot, q, dq, e_current, target, alpha=1.0, beta=0.5, max_steps=10):
    current_error_norm = np.linalg.norm(e_current)
    step_size = alpha

    for _ in range(max_steps):
        q_new = q + step_size * dq
        x_new = robot.forward_kinematics(q_new)
        e_new = error(x_new, target)

        if np.linalg.norm(e_new) < current_error_norm:
            return step_size

        step_size *= beta

    return step_size

In [86]:
def inverse_kinematics(robot, target: franky.Affine, q0):
    q = q0.copy()
    I = np.eye(7)
    k0 = 0.01
    max_iterations = 50
    tol = 1e-4; min_step = 1e-6
    pinv_reg = 0.1

    for i in range(max_iterations):
        x = robot.forward_kinematics(q)
        e = error(x, target)
        error_norm = np.linalg.norm(e)

        if error_norm < tol:
            print(f"Converged after {i} iterations")
            break

        j = robot.jacobian(q)
        j_inv = j.T @ np.linalg.inv(j @ j.T + pinv_reg * I[:6,:6])

        N = I - j_inv @ j
        dq_primary = -j_inv @ e

        k_null = k0 * np.exp(error_norm)  # Reduce null space influence when error is high
        dq_null = N @ (-k_null * q)

        dq = dq_primary + dq_null

        step_size = line_search(robot, q, dq, e, target, alpha=1.0, beta=0.8, max_steps=20)
        if step_size < min_step:
            print(f"Step size too small after {i} iterations")
            break

        q += step_size * dq

        if i % 1 == 0:  # Print less frequently
            print(f"{i}: Err: {error_norm:.5f}, Step: {step_size:.3f}, dq_p: {np.linalg.norm(dq_primary)}, dq_n: {np.linalg.norm(dq_null)}")

    return q

In [87]:
pos = robot.forward_kinematics(np.array(robot.state.q))

In [88]:
ik_q = inverse_kinematics(robot, pos, np.array(robot.state.q) + np.random.randn(7) * 0.06)
robot.forward_kinematics(ik_q), robot.forward_kinematics(np.array(robot.state.q))

0: Err: 0.10046, Step: 1.000, dq_p: 0.09043859473031989, dq_n: 0.0186720190761139
1: Err: 0.02048, Step: 1.000, dq_p: 0.03079876439263302, dq_n: 0.017190893139847493
2: Err: 0.01014, Step: 1.000, dq_p: 0.015161613969496527, dq_n: 0.016951045294983303
3: Err: 0.00761, Step: 0.410, dq_p: 0.010358871047158982, dq_n: 0.016850969104743877
4: Err: 0.00760, Step: 0.012, dq_p: 0.010013301920873383, dq_n: 0.016830451774240543
5: Err: 0.00760, Step: 0.012, dq_p: 0.01000970081647311, dq_n: 0.01682996444837253
6: Err: 0.00761, Step: 0.012, dq_p: 0.010006258684405387, dq_n: 0.016829479508533172
7: Err: 0.00761, Step: 0.012, dq_p: 0.010002973807633491, dq_n: 0.016828996933067067
8: Err: 0.00761, Step: 0.012, dq_p: 0.00999984448178529, dq_n: 0.016828516700515685
9: Err: 0.00762, Step: 0.012, dq_p: 0.009996869015155629, dq_n: 0.016828038789616646
10: Err: 0.00762, Step: 0.012, dq_p: 0.009994045728706888, dq_n: 0.016827563179302826
11: Err: 0.00762, Step: 0.012, dq_p: 0.009991372956068221, dq_n: 0.0168

(Affine(t=[0.463246 0.00181483 0.696371], q=[0.989572 -0.00401539 0.143963 0.00223078]),
 Affine(t=[0.462173 0.00470519 0.689736], q=[0.989394 -0.00466427 0.145159 0.00244009]))

In [89]:
np.linalg.norm(ik_q), np.linalg.norm(np.array(robot.state.q))

(2.2830651207484833, 2.3262172049749097)

In [85]:
np.linalg.norm(ik_q), np.linalg.norm(np.array(robot.state.q))

(2.319937667768981, 2.3262163489116574)

In [12]:
target = pos
q0 = np.array(robot.state.q) + np.random.randn(7) * 0.03
q = q0.copy()
pinv_reg, I = 0.01, np.eye(7)

In [13]:
x = robot.forward_kinematics(q)
e = error(x, target)
error_norm = np.linalg.norm(e)

In [8]:
def error(x_cur, target):
    quat = np.array(x_cur.quaternion)
    q_e = quaternion_multiply(quat, q_inv(target.quaternion))
    q_e = Rotation.from_quat([q_e[1], q_e[2], q_e[3], q_e[0]]).as_euler('ZYX')
    return np.concatenate([x_cur.translation - target.translation, q_e])

In [29]:
# _q = np.zeros(7)
_q = np.array([0, np.pi/2, 0, np.pi/2, 0, 0, 0])
_q = robot.state.q

In [30]:
eps = 1e-5
J_num = np.zeros((6, 7))

for i in range(7):
    dq = np.zeros(7)
    dq[i] = eps
    x_plus = error(robot.forward_kinematics(_q + dq), robot.forward_kinematics(_q))
    x_minus = error(robot.forward_kinematics(_q - dq), robot.forward_kinematics(_q))
    J_num[:,i] = (x_plus - x_minus) / (2 * eps)

In [31]:
J_num

array([[-0.005342,  0.455747, -0.006534, -0.12941 , -0.001178,  0.077212,  0.      ],
       [ 0.432478,  0.004777,  0.549825, -0.001316,  0.111336,  0.001924,  0.      ],
       [ 0.      , -0.43251 , -0.000244,  0.449252, -0.001072,  0.115011,  0.      ],
       [-0.205574,  0.720075,  0.003957, -0.722378, -0.690904, -0.722085,  0.      ],
       [-0.200656, -0.693835,  0.017355,  0.691438, -0.721146,  0.691804,  0.      ],
       [ 0.95785 ,  0.009194,  0.999842, -0.009143,  0.050981, -0.      , -1.      ]])

In [32]:
J = robot.jacobian(_q)
J

array([[-0.005342,  0.455747, -0.006534, -0.12941 , -0.001178,  0.077212, -0.      ],
       [ 0.432478,  0.004777,  0.549825, -0.001316,  0.111336,  0.001924,  0.      ],
       [ 0.      , -0.43251 , -0.000244,  0.449252, -0.001072,  0.115011, -0.      ],
       [ 0.      , -0.01048 , -0.301686,  0.01365 ,  0.941924,  0.010578,  0.287202],
       [ 0.      ,  0.999945, -0.003162, -0.999906,  0.013195, -0.999898, -0.006184],
       [ 1.      ,  0.      ,  0.953402,  0.001003,  0.335566,  0.009627, -0.95785 ]])

In [36]:
q1 = _q + 0.01 * np.array([0, 1, 0, 0, 0, 0, 0])

In [38]:
p1, p = robot.forward_kinematics(q1), robot.forward_kinematics(_q)
# np.array(p1.quaternion), np.array(p.quaternion)
# np.array(p1.quaternion) - np.array(p.quaternion)

In [39]:
p1, p

(Affine(t=[0.0928955 0 0.822096], q=[0.923868 0.382679 -0.00461938 -0.00191341]),
 Affine(t=[0.088 0 0.823], q=[0.92388 0.382683 0 0]))

In [40]:
p1.translation - p.translation

array([ 0.004896,  0.      , -0.000904])

In [41]:
tr_diff = quaternion_multiply(np.array(p1.quaternion), q_inv(p.quaternion))
tr_diff

array([ 0.999988, -0.      , -0.003536, -0.003536])

In [42]:
Rotation.from_quat([tr_diff[1], tr_diff[2], tr_diff[3], tr_diff[0]]).as_euler("ZYX")


array([-0.007071, -0.007071,  0.000025])

In [43]:
error(p1, p)

array([ 0.004896,  0.      , -0.000904, -0.007071, -0.007071,  0.000025])

In [44]:
np.array(J @ (q1 - _q))

array([ 0.0049 ,  0.     , -0.00088,  0.     ,  0.01   ,  0.     ])

In [26]:
Rotation.from_euler("ZYX", np.array(J @ (q1 - _q))[3:]).as_quat()

array([0.005   , 0.      , 0.      , 0.999988])

In [142]:
p.translation + np.array(J @ (q1 - _q))[:3], p1.translation

(array([0.0929 , 0.     , 0.82212]), array([0.092896, 0.      , 0.822096]))

In [173]:
Rotation.from_quat(tr_diff).as_euler("ZYX", degrees=True)


array([ -5.79177, -38.41191, 178.70059])

In [82]:
j

array([[ 0.021,  0.346,  0.02 , -0.019,  0.008,  0.174,  0.   ],
       [ 0.478,  0.   ,  0.557, -0.006,  0.21 , -0.012, -0.   ],
       [ 0.   , -0.478,  0.006,  0.49 ,  0.008,  0.147, -0.   ],
       [ 0.   , -0.   , -0.286, -0.044,  0.954, -0.037,  0.303],
       [ 0.   ,  1.   , -0.   , -0.999, -0.046, -0.999,  0.023],
       [ 1.   ,  0.   ,  0.958, -0.013,  0.297, -0.036, -0.953]])

In [85]:
j_inv @ e

array([ 0.005,  0.036,  0.007,  0.072,  0.003, -0.06 ,  0.009])

---

In [13]:
pos = robot.current_cartesian_state.pose.end_effector_pose
q = np.array(robot.state.q)

In [14]:
pos

Affine(t=[0.462171 0.00470396 0.689737], q=[0.989394 -0.00466179 0.145158 0.00244112])

In [15]:
a = np.array(robot.state.O_T_EE)
franky.Affine(a.reshape(4,4).T)

Affine(t=[0.462171 0.00470529 0.689736], q=[0.989395 -0.00466086 0.145156 0.00244099])

In [16]:
robot.forward_kinematics(q)

Affine(t=[0.462172 0.00470279 0.689735], q=[0.989395 -0.00466275 0.145156 0.00244152])

In [17]:
inverse_kinematics(robot, pos, np.array(robot.state.q))

0: Error: 0.287, Step size: 0.001, Q: [0.01, -0.31, 0.00, -1.53, -0.01, 1.52, 0.81]
5: Error: 0.288, Step size: 0.001, Q: [0.01, -0.31, 0.00, -1.53, -0.01, 1.52, 0.81]
10: Error: 0.289, Step size: 0.001, Q: [0.01, -0.31, 0.00, -1.53, -0.01, 1.52, 0.81]
15: Error: 0.289, Step size: 0.001, Q: [0.01, -0.31, 0.00, -1.53, -0.01, 1.52, 0.81]
20: Error: 0.290, Step size: 0.001, Q: [0.01, -0.31, 0.00, -1.53, -0.01, 1.52, 0.81]
25: Error: 0.290, Step size: 0.001, Q: [0.01, -0.31, 0.00, -1.53, -0.01, 1.52, 0.81]
30: Error: 0.291, Step size: 0.001, Q: [0.01, -0.31, 0.00, -1.53, -0.01, 1.52, 0.81]
35: Error: 0.292, Step size: 0.001, Q: [0.01, -0.31, 0.00, -1.53, -0.01, 1.52, 0.81]
40: Error: 0.292, Step size: 0.001, Q: [0.01, -0.31, 0.00, -1.53, -0.01, 1.52, 0.81]
45: Error: 0.293, Step size: 0.001, Q: [0.01, -0.31, 0.00, -1.53, -0.01, 1.53, 0.81]
50: Error: 0.293, Step size: 0.001, Q: [0.01, -0.31, 0.00, -1.53, -0.01, 1.53, 0.81]
55: Error: 0.294, Step size: 0.001, Q: [0.01, -0.31, 0.00, -1.53, -

array([ 0.00932885, -0.30852162,  0.00335122, -1.52478114, -0.00997415,
        1.53232885,  0.8051471 ])

In [65]:
robot.forward_kinematics(robot.state.q)

Affine(t=[0.432476 0.00534272 0.788774], q=[0.912297 -0.382936 0.135037 -0.0532912])

In [16]:
robot.state.q

[0.010480232632723234,
 -0.30647963334062095,
 0.0033271352274554856,
 -1.535063607684307,
 -0.00915653893903261,
 1.5197909502698166,
 0.8068179199968182]